<a href="https://colab.research.google.com/github/dnaae/minor-internship/blob/FISH-QUANT-pipeline/Synthetic_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install big-fish

In [ ]:
!pip install sim-fish

In [ ]:
import os
import numpy as np
import tifffile as tiff
from google.colab import drive
import bigfish
import simfish as sim
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.plot as plot
from google.colab import drive
import matplotlib.pyplot as plt
import re

In [ ]:
# Function to mount Google Drive and select a folder
def choose_folder_colab():
    drive.mount('/content/drive')
    drive_folder = "/content/drive/MyDrive/"
    contents = os.listdir(drive_folder)

    print("Contents of your Google Drive:")
    for i, item in enumerate(contents):
        print(f"{i + 1}: {item}")

    while True:
        choice = input("Enter the number of the folder you want to select (q to quit): ")
        if choice.lower() == 'q':
            return None
        try:
            choice = int(choice)
            if 1 <= choice <= len(contents):
                selected_folder = os.path.join(drive_folder, contents[choice - 1])
                return selected_folder
            else:
                print("Invalid choice. Please try again or enter 'q' to quit.")
        except (ValueError, KeyboardInterrupt):
            print("Invalid input. Please try again or enter 'q' to quit.")

In [ ]:
def process_tiff_image(image_data, channel, cell_name):
    # Perform processing on the selected FISH image
    rna_mip = stack.maximum_projection(image_data)
    rna_mip = stack.gaussian_filter(rna_mip, sigma=5)
    rna_log = stack.log_filter(rna_mip, sigma=3)
    rna_background_mean = stack.remove_background_mean(rna_mip, kernel_shape="square", kernel_size=31)
    rna_background_gaussian = stack.remove_background_gaussian(rna_mip, sigma=3)

    print(f"Processing image (Channel {channel})")
    print("smfish channel")
    print("\r shape: {0}".format(image_data.shape))
    print("\r dtype: {0}".format(image_data.dtype))
    print("smfish channel (2D maximum projection)")
    print("\r shape: {0}".format(rna_mip.shape))
    print("\r dtype: {0}".format(rna_mip.dtype))

    spot_detection(image_data, rna_mip, channel, cell_name)  # Remove path_input argument

In [ ]:
spots_post_clustering = None
clusters = None

def spot_detection(rna, rna_mip, channel, cell_name):
    spots, threshold = detection.detect_spots(
        images=rna,
        return_threshold=True,
        log_kernel_size=(1.167, 1.456, 1.456),
        minimum_distance=(1.167, 1.456, 1.456)
    )

    print("detected spots")
    print("\r shape: {0}".format(spots.shape))
    print("\r dtype: {0}".format(spots.dtype))
    print("\r threshold: {0}".format(threshold))

    # Plot spot detection
    plot.plot_detection(rna_mip, spots, contrast=True)

    # Plot elbow curve
    plot.plot_elbow(
        images=rna,
        voxel_size=(300, 103, 103),
        spot_radius=(350, 150, 150)
    )

    # Dense region decomposition
    spots_post_decomposition, dense_regions, reference_spot = detection.decompose_dense(
        image=rna,
        spots=spots,
        voxel_size=(300, 103, 103),
        spot_radius=(350, 150, 150),
        alpha=0.6,  # 0.6
        beta=1,  # 1
        gamma=5
    )

    print("detected spots before decomposition")
    print("\r shape: {0}".format(spots.shape))
    print("\r dtype: {0}".format(spots.dtype), "\n")
    print("detected spots after decomposition")
    print("\r shape: {0}".format(spots_post_decomposition.shape))
    print("\r dtype: {0}".format(spots_post_decomposition.dtype))

    # Plot spot detection after decomposing regions
    plot.plot_detection(rna_mip, spots_post_decomposition, contrast=True)

    # Clusters detection
    global spots_post_clustering
    global clusters
    spots_post_clustering, clusters = detection.detect_clusters(
        spots=spots_post_decomposition,
        voxel_size=(100, 103, 103),
        radius=1000,
        nb_min_spots=2  # 4
    )

    print("detected spots after clustering")
    print("\r shape: {0}".format(spots_post_clustering.shape))
    print("\r dtype: {0}".format(spots_post_clustering.dtype), "\n")
    print("detected clusters")
    print("\r shape: {0}".format(clusters.shape))
    print("\r dtype: {0}".format(clusters.dtype))

    # Plot
    plot.plot_detection(
        rna_mip,
        spots=[spots_post_decomposition, clusters[:, :3]],
        shape=["circle", "polygon"],
        radius=[3, 6],
        color=["red", "blue"],
        linewidth=[1, 2],
        fill=[False, True],
        contrast=True
    )


In [ ]:
def process_images_interactively(image_generator):
    # Iterate over all images in the generator
    for index, image_tuple in enumerate(image_generator):
        image_data = image_tuple[0]  # Extract the image data from the tuple
        print(f"Processing Image {index + 1}")

        # Perform processing on the selected FISH image
        process_tiff_image(image_data, 2, f"cell_{index}")

        # Check if spots and clusters are detected
        if spots_post_clustering is not None:
            # Save spots data to CSV file
            spots_file_path = f"spots_cell_{index}.csv"
            stack.save_data_to_csv(spots_post_clustering, spots_file_path)
            print(f"CSV file saved for Spots - cell_{index}")

        if clusters is not None:
            # Save clusters data to CSV file
            clusters_file_path = f"clusters_cell_{index}.csv"
            stack.save_data_to_csv(clusters, clusters_file_path)
            print(f"CSV file saved for Clusters - cell_{index}")



# Generate synthetic image data
image_generator = sim.simulate_images(n_images=1, ndim=3, n_spots=100,n_clusters=10,

                                      image_shape=(15,512,512), image_dtype=np.uint16,
                                voxel_size=(100, 100,100), sigma=(100, 100,100),
                                      )

# Call process_images_interactively with synthetic data
process_images_interactively(image_generator)


In [ ]:
#test
def process_images_interactively(image_generator):
    # Iterate over all images in the generator
    for index, image_tuple in enumerate(image_generator):
        image_data = image_tuple[0]  # Extract the image data from the tuple
        print(f"Processing Image {index + 1}")

        # Perform processing on the selected FISH image
        process_tiff_image(image_data, 2, f"cell_{index}")

        # Save the data to CSV files
        try:
            if spots_post_clustering is not None:
                stack.save_data_to_csv(spots_post_clustering, f"spots_cell_{index}.csv")
                print(f"CSV file saved for Spots - cell_{index}")
            if clusters is not None:
                stack.save_data_to_csv(clusters, f"clusters_cell_{index}.csv")
                print(f"CSV file saved for Clusters - cell_{index}")
        except Exception as e:
            print(f"Error saving CSV files for cell_{index}: {e}")


# Generate synthetic image data
image_generator = sim.simulate_images(n_images=3, ndim=3, n_spots=100, random_n_spots=False, n_clusters=25,
                                      random_n_clusters=False, n_spots_cluster=0, random_n_spots_cluster=False,
                                      centered_cluster=False, image_shape=(128, 128,128), image_dtype=np.uint16,
                                      subpixel_factors=None, voxel_size=(100, 100,100), sigma=(150, 150,150),
                                      random_sigma=0.05, amplitude=5000, random_amplitude=0.05,
                                      noise_level=300, random_noise=0.05)

# Call process_images_interactively with synthetic data
process_images_interactively(image_generator)